In [4]:
#!conda install -q -u ulmo
!mamba install -q -y ulmo

  Package            Version  Build              Channel                    Size
──────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────────────────

  beautifulsoup4      4.10.0  pyha770c72_0       conda-forge/noarch        77 KB
  future              0.18.2  py39hf3d152e_4     conda-forge/linux-64     715 KB
  isodate              0.6.1  pyhd8ed1ab_0       conda-forge/noarch        28 KB
  libxslt             1.1.33  h0ef7038_3         conda-forge/linux-64     521 KB
  lxml                 4.8.0  py39h107f48f_0     conda-forge/linux-64       2 MB
  soupsieve            2.3.1  pyhd8ed1ab_0       conda-forge/noarch        33 KB
  suds-jurko             0.6  py39hf3d152e_1005  conda-forge/linux-64     168 KB
  ulmo                 0.8.8  pyh6c4a22f_0       conda-forge/noarch        63 KB

  Upgrade:
─────────────────────────────────────────────────────────────────────────────────

In [5]:
import ulmo

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [11]:
site_list = ['SNOTEL:1011_WA_SNTL', 'SNOTEL:909_WA_SNTL', 'SNOTEL:999_WA_SNTL', 'SNOTEL:910_WA_SNTL']

In [6]:
#http://his.cuahsi.org/wofws.html
wsdlurl = 'https://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'

In [7]:
today = pd.to_datetime("today")
today.strftime('%Y-%m-%d')

'2022-03-08'

In [8]:
sites = ulmo.cuahsi.wof.get_sites(wsdlurl)

In [10]:
sites_df = pd.DataFrame.from_dict(sites, orient='index').dropna()
sites_df.head()

,code,name,network,location,elevation_m,site_property
SNOTEL:301_CA_SNTL,301_CA_SNTL,Adin Mtn,SNOTEL,"{'latitude': '41.2358283996582', 'longitude': ...",1886.7120361328125,"{'county': 'Modoc', 'state': 'California', 'si..."
SNOTEL:907_UT_SNTL,907_UT_SNTL,Agua Canyon,SNOTEL,"{'latitude': '37.522171020507813', 'longitude'...",2712.719970703125,"{'county': 'Kane', 'state': 'Utah', 'site_comm..."
SNOTEL:916_MT_SNTL,916_MT_SNTL,Albro Lake,SNOTEL,"{'latitude': '45.59722900390625', 'longitude':...",2529.840087890625,"{'county': 'Madison', 'state': 'Montana', 'sit..."
SNOTEL:1267_AK_SNTL,1267_AK_SNTL,Alexander Lake,SNOTEL,"{'latitude': '61.749668121337891', 'longitude'...",48.768001556396484,"{'county': 'Matanuska-Susitna', 'state': 'Alas..."
SNOTEL:908_WA_SNTL,908_WA_SNTL,Alpine Meadows,SNOTEL,"{'latitude': '47.779571533203125', 'longitude'...",1066.800048828125,"{'county': 'King', 'state': 'Washington', 'sit..."


In [12]:
baker_snotel = sites_df.loc[ sites_df.index.isin(site_list), : ]

In [15]:
baker_snotel['geometry'] = [Point(float(l['longitude']), float(l['latitude'])) for l in baker_snotel['location']]

/srv/conda/envs/notebook/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/tmp/ipykernel_342/975464778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baker_snotel['geometry'] = [Point(float(l['longitude']), float(l['latitude'])) for l in baker_snotel['location']]


In [16]:
baker_snotel = baker_snotel.drop(columns='location')
baker_snotel = baker_snotel.astype({"elevation_m":float})

In [17]:
baker_snotel_gdf = gpd.GeoDataFrame(baker_snotel, crs='EPSG:4326')
baker_snotel_gdf.shape

(4, 6)

In [18]:
baker_snotel = 'snotel_conus_baker.json'
if not os.path.exists(baker_snotel):
    baker_snotel_gdf.to_file(baker_snotel, driver='GeoJSON')

In [26]:
#Get current datetime
today = pd.to_datetime("today").strftime('%Y-%m-%d')

def snotel_fetch(sitecode, variablecode='SNOTEL:WTEQ_D', start_date='1950-10-01', end_date=today):
    #print(sitecode, variablecode, start_date, end_date)
    values_df = None
    try:
        #Request data from the server
        site_values = ulmo.cuahsi.wof.get_values(wsdlurl, sitecode, variablecode, start=start_date, end=end_date)
    except:
        print(f"Unable to fetch {variablecode} for {sitecode}")
    else:
        #Convert to a Pandas DataFrame   
        values_df = pd.DataFrame.from_dict(site_values['values'])
        #Parse the datetime values to Pandas Timestamp objects
        values_df['datetime'] = pd.to_datetime(values_df['datetime'], utc=True)
        #Set the DataFrame index to the Timestamps
        values_df = values_df.set_index('datetime')
        #Convert values to float and replace -9999 nodata values with NaN
        values_df['value'] = pd.to_numeric(values_df['value']).replace(-9999, np.nan)
        #Remove any records flagged with lower quality
        values_df = values_df[values_df['quality_control_level_code'] == '1']

    return values_df

In [27]:
#Isolated to WA SNOTEL sites
pkl_fn = 'snotel_wteq_d_baker.pkl'
gdf = baker_snotel_gdf

In [28]:
variablecode = 'SNOTEL:WTEQ_D'
#variablecode = 'SNOTEL:SNWD_D'

In [29]:
%%time
if os.path.exists(pkl_fn):
    snwd_df = pd.read_pickle(pkl_fn)
else:
    #Define an empty dictionary to store returns for each site
    value_dict = {}
    #Loop through each site and add record to dictionary
    for i, sitecode in enumerate(gdf.index):
        print('%i of %i sites: %s' % (i+1, len(gdf.index), sitecode))
        #out = snotel_fetch(sitecode, variablecode, start_date, end_date)
        out = snotel_fetch(sitecode, variablecode)
        if out is not None:
            value_dict[sitecode] = out['value']
    
    #Convert the dictionary to a DataFrame, automatically handles different datetime ranges (nice!)
    snwd_df = pd.DataFrame.from_dict(value_dict)
    #Write out
    print(f"Writing out: {pkl_fn}")
    snwd_df.to_pickle(pkl_fn)

1 of 4 sites: SNOTEL:910_WA_SNTL
2 of 4 sites: SNOTEL:999_WA_SNTL
3 of 4 sites: SNOTEL:1011_WA_SNTL
4 of 4 sites: SNOTEL:909_WA_SNTL
Writing out: snotel_wteq_d_baker.pkl
CPU times: user 3.72 s, sys: 26.4 ms, total: 3.75 s
Wall time: 13.2 s
